In [ ]:
!pip3 install -q transformers

#Pretraining

In [ ]:
import pandas as pd

In [ ]:
from LMkor.examples.bertshared_summarization import Summarize
summarize = Summarize('/bertshared-kor-base')

In [ ]:
from transformers import BertTokenizerFast, EncoderDecoderModel
model_bertshared = EncoderDecoderModel.from_pretrained("/bertshared-kor-base")

In [ ]:
text = '''
LG전자가 스마트폰을 담당하는 MC(모바일커뮤니케이션)사업부 분할 및 매각을 위한 법률 자문 업무를 김앤장법률사무소에 맡겼다. MC사업부 매각 작업에 속도가 붙을지 관심이 집중되고 있다.
22일 인수합병(M&A)업계에 따르면 LG전자는 최근 MC사업부 분할 후 매각 방안 등을 포괄적으로 검토하기 위해 김앤장을 법률자문사로 선임한 것으로 알려졌다. 회계·실사 자문은 EY한영회계법인에 맡길 가능성이 큰 것으로 전해졌다. 김앤장 등 자문사들은 사업본부를 분할한 뒤 사업양수도나 분할사업부의 지분 매각, 지식재산권(IP) 매각 등을 놓고 검토에 들어간 것으로 알려졌다.
업계에서는 LG전자가 MC사업본부를 통매각하기보다는 ‘쪼개기 매각’에 나설 것으로 보고 있다. 스마트폰 선행기술 연구개발(R&D) 등 핵심 기능만 남겨둔 채 매각을 시도할 것으로 관측하고 있다. 앞서 권봉석 LG전자 사장은 사내 메시지를 통해 임직원에게 “현재 모든 가능성을 열어 두고 사업 운영방향을 면밀히 검토하고 있다”고 밝히며 매각 추진을 암시했다. M&A업계 관계자는 “거래가 성사되기도 전에 사업 전면 재검토를 공식화한 것은 상당히 이례적”이라며 “향후 매각이 잘 이뤄지지 않더라도 모바일 사업을 철수하겠다는 배수진을 둔 것으로 보인다”고 설명했다.
다만 원매자를 찾기가 쉽지 않을 것이란 전망이 우세하다. LG전자 모바일 사업은 한때 글로벌시장에서 톱5 안에 드는 기술력을 인정받았지만 누적 적자만 5조원에 달하고 있다. 업계에서 평가하는 MC사업부의 가치도 5000억원대에서 수조원대까지 편차가 상당히 크다.
상대적으로 해외 원매자들의 인수의사가 더 확실한 것으로 알려지고 있다. 북미사업 등 글로벌 시장 확장을 원하는 후발기업들이 주요 대상이다. 베트남의 빈그룹과 중국 기업 등이 유력하게 거론된다. 증권업계를 중심으로는 스마트 기기를 연결하는 사물인터넷(IoT) 사업을 염두에 둔 구글, 페이스북 같은 미국 정보기술(IT) 기업들도 원매자 후보군으로 꼽고 있다.
'''

summarize(text)

In [ ]:
type(summarize(text))

In [ ]:
a = summarize(text)
print(a)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
news23 = pd.read_csv("/content/drive/MyDrive/2022-08-23.csv")

In [ ]:
news23.head()

In [ ]:
for i in range(len(news23)):
  passage = news23['text'][i]
  print(len(passage))

In [ ]:
pretrain_sum = []
for i in range(len(news23)):
  passage = news23['text'][i]
  try:
    sum = str(summarize(passage))
    pretrain_sum.append(sum)
  except :
    pretrain_sum.append('')

In [ ]:
pretrain_sum[0]

#Dataset

In [ ]:
pip install datasets

In [ ]:
from datasets import list_datasets, load_dataset
from datasets import load_dataset

In [ ]:
import os
import shutil
import json
from tqdm import tqdm
import numpy as np
import pandas as pd

In [ ]:
!pip install transformers

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/train.csv')

In [ ]:
data = df['passage']
print(len(data))
data.head()

In [ ]:
summary = df['summary']

In [ ]:
import transformers
from transformers import BertTokenizerFast, EncoderDecoderModel
transformers.logging.set_verbosity_error()

In [ ]:
tokenizer =BertTokenizerFast.from_pretrained("/bertshared-kor-base")

In [ ]:
tokenizer.bos_token = tokenizer.cls_token
tokenizer.eos_token = tokenizer.sep_token

In [ ]:
tokenizer.bos_token = 2
tokenizer.eos_token = 3

In [ ]:
df.drop(['Unnamed: 0'],axis=1,inplace=True)
df.head()

In [ ]:
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader, random_split, RandomSampler, SequentialSampler
torch.manual_seed(42)
import random

from transformers import GPT2LMHeadModel, GPT2Config
from transformers import AdamW, get_linear_schedule_with_warmup

In [ ]:
model = EncoderDecoderModel.from_pretrained("/bertshared-kor-base")

In [ ]:
import pandas as pd
import torch.utils.data as data_utils

In [ ]:
import datasets

In [ ]:
passage = []
summary = []
for i in range(4000):
  passage.append(df['passage'][i])
  summary.append(df['summary'][i])

train_dict = {'passage':passage, 'summary':summary}

train_set = datasets.Dataset.from_dict(train_dict)

In [ ]:
batch_size=4  # change to 16 for full training
encoder_max_length=512
decoder_max_length=128

def process_data_to_model_inputs(batch):
  # tokenize the inputs and labels
  inputs = tokenizer(batch["passage"], padding="max_length", truncation=True, max_length=encoder_max_length)
  outputs = tokenizer(batch["summary"], padding="max_length", truncation=True, max_length=decoder_max_length)

  batch["input_ids"] = inputs.input_ids
  batch["attention_mask"] = inputs.attention_mask
  batch["decoder_input_ids"] = outputs.input_ids
  batch["decoder_attention_mask"] = outputs.attention_mask
  batch["labels"] = outputs.input_ids.copy()

  # because BERT automatically shifts the labels, the labels correspond exactly to `decoder_input_ids`. 
  # We have to make sure that the PAD token is ignored
  batch["labels"] = [[-100 if token == tokenizer.pad_token_id else token for token in labels] for labels in batch["labels"]]

  return batch

# only use 32 training examples for notebook - DELETE LINE FOR FULL TRAINING
#train_data = train_data.select(range(32))

train_data = train_set.map(
    process_data_to_model_inputs, 
    batched=True, 
    batch_size=batch_size, 
    remove_columns=["passage", "summary"]
)
train_data.set_format(
    type="torch", columns=["input_ids", "attention_mask", "decoder_input_ids", "decoder_attention_mask", "labels"],
)


# only use 16 training examples for notebook - DELETE LINE FOR FULL TRAINING
#val_data = val_set.select(range(16))

val_data = val_set.map(
    process_data_to_model_inputs, 
    batched=True, 
    batch_size=batch_size, 
    remove_columns=["passage", "summary"]
)
val_data.set_format(
    type="torch", columns=["input_ids", "attention_mask", "decoder_input_ids", "decoder_attention_mask", "labels"],
)

In [ ]:
train_data[4]

#Finetuning

In [ ]:
model.config.max_length = 512
model.config.min_length = 128

In [ ]:
# set special tokens
model.config.decoder_start_token_id = tokenizer.bos_token_id
model.config.eos_token_id = tokenizer.eos_token_id
model.config.pad_token_id = 0 #tokenizer.pad_token_id

# sensible parameters for beam search
model.config.vocab_size = 42000 #model.config.decoder.vocab_size
model.config.max_length = 40
model.config.min_length = 0
model.config.no_repeat_ngram_size = 0
model.config.early_stopping = False
model.config.length_penalty = 1.0
model.config.num_beams = 1

In [ ]:
model.config.vocab_size = 42000 #model.config.decoder.vocab_size
model.config.max_length = 100
model.config.min_length = 10

In [ ]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer

In [ ]:
from datasets import load_dataset, load_metric

In [ ]:
pip install rouge_score

In [ ]:
# load rouge for validation
rouge = datasets.load_metric("rouge")

def compute_metrics(pred):
    labels_ids = pred.label_ids
    pred_ids = pred.predictions

    # all unnecessary tokens are removed
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    labels_ids[labels_ids == -100] = tokenizer.pad_token_id
    label_str = tokenizer.batch_decode(labels_ids, skip_special_tokens=True)

    rouge_output = rouge.compute(predictions=pred_str, references=label_str, rouge_types=["rouge2"])["rouge2"].mid

    return {
        "rouge2_precision": round(rouge_output.precision, 4),
        "rouge2_recall": round(rouge_output.recall, 4),
        "rouge2_fmeasure": round(rouge_output.fmeasure, 4),
    }

In [ ]:
model.to("cuda")

In [ ]:
# set training arguments - these params are not really tuned, feel free to change
training_args = Seq2SeqTrainingArguments(
    output_dir="./",
    evaluation_strategy="steps",
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    predict_with_generate=True,
    logging_steps=2,  # set to 1000 for full training
    save_steps=8,  # set to 500 for full training
    eval_steps=4,  # set to 8000 for full training
    warmup_steps=1,  # set to 2000 for full training
    max_steps=16, # delete for full training
    overwrite_output_dir=True,
    save_total_limit=3,
    #fp16=True, 
)

# instantiate trainer
trainer = Seq2SeqTrainer(
    model=model,
    tokenizer=tokenizer,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_data,
    eval_dataset=val_data,
)
trainer.train()

In [ ]:
/content/checkpoint-16

In [ ]:
torch.save(model, '/content/drive/MyDrive/model3.pt')

In [ ]:
trained_model = torch.load('/content/drive/MyDrive/model3.pt') 

In [ ]:
text = '''
LG전자가 스마트폰을 담당하는 MC(모바일커뮤니케이션)사업부 분할 및 매각을 위한 법률 자문 업무를 김앤장법률사무소에 맡겼다. MC사업부 매각 작업에 속도가 붙을지 관심이 집중되고 있다.
22일 인수합병(M&A)업계에 따르면 LG전자는 최근 MC사업부 분할 후 매각 방안 등을 포괄적으로 검토하기 위해 김앤장을 법률자문사로 선임한 것으로 알려졌다. 회계·실사 자문은 EY한영회계법인에 맡길 가능성이 큰 것으로 전해졌다. 김앤장 등 자문사들은 사업본부를 분할한 뒤 사업양수도나 분할사업부의 지분 매각, 지식재산권(IP) 매각 등을 놓고 검토에 들어간 것으로 알려졌다.
업계에서는 LG전자가 MC사업본부를 통매각하기보다는 ‘쪼개기 매각’에 나설 것으로 보고 있다. 스마트폰 선행기술 연구개발(R&D) 등 핵심 기능만 남겨둔 채 매각을 시도할 것으로 관측하고 있다. 앞서 권봉석 LG전자 사장은 사내 메시지를 통해 임직원에게 “현재 모든 가능성을 열어 두고 사업 운영방향을 면밀히 검토하고 있다”고 밝히며 매각 추진을 암시했다. M&A업계 관계자는 “거래가 성사되기도 전에 사업 전면 재검토를 공식화한 것은 상당히 이례적”이라며 “향후 매각이 잘 이뤄지지 않더라도 모바일 사업을 철수하겠다는 배수진을 둔 것으로 보인다”고 설명했다.
다만 원매자를 찾기가 쉽지 않을 것이란 전망이 우세하다. LG전자 모바일 사업은 한때 글로벌시장에서 톱5 안에 드는 기술력을 인정받았지만 누적 적자만 5조원에 달하고 있다. 업계에서 평가하는 MC사업부의 가치도 5000억원대에서 수조원대까지 편차가 상당히 크다.
상대적으로 해외 원매자들의 인수의사가 더 확실한 것으로 알려지고 있다. 북미사업 등 글로벌 시장 확장을 원하는 후발기업들이 주요 대상이다. 베트남의 빈그룹과 중국 기업 등이 유력하게 거론된다. 증권업계를 중심으로는 스마트 기기를 연결하는 사물인터넷(IoT) 사업을 염두에 둔 구글, 페이스북 같은 미국 정보기술(IT) 기업들도 원매자 후보군으로 꼽고 있다.
'''

In [ ]:
trained_model.to("cuda")

In [ ]:
input_ids = tokenizer.encode(text, return_tensors= 'pt').to("cuda")

sentence_length = len(input_ids[0])
min_length = max(10, int(0.1*sentence_length))
max_length = min(128, int(0.3*sentence_length))

outputs = trained_model.generate(
            input_ids,
            min_length=min_length,
            max_length=max_length
        ).to("cuda")


print(tokenizer.decode(outputs[0], skip_special_tokens=True))

In [ ]:
df2 = pd.read_csv("/content/drive/MyDrive/test.csv")

df2.drop(['Unnamed: 0'],axis=1,inplace=True)
passage = []
summary = []
for i in range(len(df2)):
  passage.append(df2['passage'][i])
  summary.append(df2['summary'][i])

test_dict = {'passage':passage, 'summary':summary}

test_set = datasets.Dataset.from_dict(test_dict)

In [ ]:
import torch
import gc
gc.collect()
torch.cuda.empty_cache()

In [ ]:
import datasets
from transformers import BertTokenizer, EncoderDecoderModel

tokenizer =BertTokenizerFast.from_pretrained("/bertshared-kor-base")
model = EncoderDecoderModel.from_pretrained("./checkpoint-16")
model.to("cuda")



batch_size = 2 


def generate_summary(batch):
    # cut off at BERT max length 512
    inputs = tokenizer(batch["article"], padding="max_length", truncation=True, max_length=512, return_tensors="pt")
    input_ids = inputs.input_ids.to("cuda")
    attention_mask = inputs.attention_mask.to("cuda")

    outputs = model.generate(input_ids, attention_mask=attention_mask)

    output_str = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    batch["pred_summary"] = output_str

    return batch

In [ ]:
import datasets
from transformers import BertTokenizer, EncoderDecoderModel

tokenizer =BertTokenizerFast.from_pretrained("kykim/bertshared-kor-base")
model = EncoderDecoderModel.from_pretrained("./checkpoint-16")
model.to("cuda")

batch_size = 2  # change to 64 for full evaluation

In [ ]:
text = '''
현대자동차가 미국 '인플레이션 감축법' 대응 방안으로 미국 조지아주에 설립키로 한 전기차 전용공장의 착공 시점을 앞당기기로 했다. 또 전기차 보조금만큼의 가격 할인도 검토 중이다.
23일 완성차 업계에 따르면 현대차는 당초 내년 상반기 착공 예정이었던 조지아주 전기차 전용공장 설립을 올해 안에 시작하기로 했다.
현대차가 올해 공사를 시작하면 2024년 하반기엔 공장을 완성할 수 있다.
이 같은 결정은 인플레이션 감축법으로 인해 받지 못하는 전기차 보조금 때문에 시장점유율 이탈을 우려한 때문이다.
현대차그룹은 현재 미국 시장에서 아이오닉5, 코나EV, 제네시스 GV60, EV6, 니로EV 등 5개 모델를 판매 중이다.
현대차는 미국 내 전기차 조립 라인이 없어 5개 모델 모두를 국내에서 생산해 수출하고 있다.
현대차는 GV70 전기차와 EV9 등 일부 차종은 기존 미국 생산 라인을 전환해 현지 생산할 계획이지만, 주력 차종인 아이오닉5 등은 여전히 국내에서 만들어 미국으로 보내야 하는 상황이다.
정부의 전기차 보조금 지급 대상에서 제외된다. 현대차가 조금씩 넓히고 있는 전기차 시장에 빨간불이 켜진 셈이다.
현대차는 올해 상반기 미국 시장 전기차 점유율에서 테슬라(70%)에 이어 2위(9%)를 차지했다.
현대차는 비록 2위지만 올해 들어 지난달까지 약 4만대의 전기차를 미국에 수출하는 등 미국 포드와 독일 폭스바겐을 제치며 성장 중이었다.
하지만 보조금을 받지 못해 가격이 오르면 경쟁력에서 밀리고 결국 시장 선점에 어려움을 겪을 수 있다.
실제로 아이오닉 5의 가격은 보조금 7500달러(약 1000만원)을 제외하면 4만 달러(약 5250만원) 수준이다.
비슷한 성능인 포드의 머스탱 마하E는 4만4000달러(5800만원)로 아이오닉 5보다 500만원가량 비쌌다. 하지만 보조금을 받지 못하면 포드 머스탱 마하E가 아이오닉 5보다 450만원 정도 가격이 낮아진다.
이에 따라 현대차그룹은 이번 인플레이션 감축법 문제로 조지아주 공장 설립 시기를 앞당기고 또 한편으로는 일정기간 가격 할인 등의 프로모션(판촉활동)도 검토 중인 것으로 알려졌다.
현대차에서 소비자들에게 가격을 보전해준다면 수익성은 줄어들겠지만 시장점유율은 확보할 수 있다. 전기차 시장에선 점유율이 중요한 만큼 현대차의 고심이 깊어지는 것이다.
완성차업계 관계자는 "연내 조지아주 공장 착공을 목표로 알고 있다"며 "그 외에도 여러가지 방법을 검토 중"이라고 했다.
한편 조 바이든 미국 대통령이 서명해 시행된 인플레이션 감축법은 북미에서 최종 조립되는 전기차만 보조금을 받을 수 있도록 하고 있다. 이 법으로 인해 전기차 보조금 지급 대상이 기존 72개 모델에서 21개로 축소됐다.
지급 대상 차종은 아우디 Q5, BMW X5와 3시리즈 플러그인, 포드Mach-E, F 시리즈, 에스케이프 PHEV와 Transit 밴, 크라이슬러 Pacifica PHEV, 지프 그랜드 체로키 PHEV, 랭글러 PHEV, 링컨 에비에이터 PHEV, 코세어 플러그인, 루시드 에어, 닛산 리프, 볼보 S60, 리비안 R1S와 R1T 등이다. 테슬라와 GM 전기차도 받는다.
반면 현대차그룹, 포르쉐 등이 판매하는 전기차는 보조금을 받을 수 없게 된다.
'''

In [ ]:
generate_summary(test_set[10])

In [ ]:
sentence_length

In [ ]:
input_ids = tokenizer.encode(text, return_tensors= 'pt',padding="max_length",truncation=True, max_length = encoder_max_length)

sentence_length = len(input_ids[0])
min_length = max(10, int(0.1*sentence_length))
max_length = min(128, int(0.3*sentence_length))


outputs = model.generate(
            input_ids,
            min_length=min_length,
            max_length=max_length
        )

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

In [ ]:
torch.save(model, '/content/drive/MyDrive/model2.pt')

In [ ]:
/content/checkpoint-16
/content/runs